# Movies Recommend System

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# 明日すること

## MEANで全体を割ったもので制度を計算してみる

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

レビュー類似度を用いた Movies Recommend System の開発

使用する映画レビューサイトは　http://www.jtnews.jp/index.html

### 構成

1. スクレイピング/SQL
2. レビューの形態素解析
3. 特徴量行列の作成
4. モデル作成

### 準備

In [1]:
#パンダスの準備
import pandas as pd
from pandas import Series,DataFrame

In [2]:
#SQLの準備
import sqlite3
con = sqlite3.connect("movies.db")

In [3]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

### 1. スクレイピング/SQL

In [4]:
sql_query = '''
SELECT * FROM reviews
'''
reviews_df = sql_to_df(sql_query)

In [5]:
reviews_df

,id,title,review
0,1,愛は静けさの中に,35．《ネタバレ》 ヒロインのマーリー・マトリンは、イザベル・アジャーニみたいに美人だけど、...
1,2,愛は静けさの中に,34．昔ながらのよくある正統派のラブストーリーだが、聾学校という舞台背景が設定の妙味に。ヒロ...
2,3,愛は静けさの中に,33．耳が聞こえる男と耳が聞こえない女は、どうすれば一つの同じ世界に生きることができるだろう...
3,4,愛は静けさの中に,32．生徒や恋人、聾唖者との関わりを重視したドラマだと思っていたが、よくある一組のカップルの...
4,5,愛は静けさの中に,31．《ネタバレ》 二人の恋のゆくえより、生徒とのやりとりのほうが面白かった。生徒達はかわい...
5,6,愛は静けさの中に,30．《ネタバレ》 マーリー・マトリン美しい… 賞受賞も納得の内容で演出面も素晴らしい！ ...
6,7,愛は静けさの中に,29．教師役として抑えた演技が光るウィリアム・ハートと、この作品でオスカーを獲得した女優マー...
7,8,愛は静けさの中に,28．自分が好きだった人と、ヒロインが同じ髪型（「フラッシュダンス」のジェニファービールスと...
8,9,愛は静けさの中に,27．こんな純愛ドラマがあったなぁとか思いながら鑑賞しました。ストーリーには凝ったところもな...
9,10,愛は静けさの中に,26．ウィリアム・ハートとマーリー・マトリン、この二人の演技がすごい。特にマーリー・マトリン...


In [6]:
hiku_df = reviews_df.ix[reviews_df['review'].str.contains('"')]

In [7]:
hiku_df

,id,title,review
21,22,愛は静けさの中に,14． 原作はマーク・メドフの舞台劇「小さき神の子等」。ヒロインのマーリー・マトリンは舞台か...
480,481,アビス,"129．Through a series named as ""See It Big! Sci..."
818,819,アポロ１３,"252．""アポロ13号""。以前、原作を本で読んでいたく感動しました。さて映画はというと、良か..."
912,913,アマデウス,"369．空想のストーリーとしてはなかなか面白いですよ。真実など誰も知らないと前提して。""だっ..."
1163,1164,アラジン,"114．《ネタバレ》 作品のテーマは""自由""。映画には３人の自由を求めるキャラクターがいる。..."
1277,1278,アリス(1990),"14．""金満夫人の憂鬱な日々""といった内容の女性映画。セレブにはセレブなりの悩みや不満がある..."
1295,1296,ある貴婦人の肖像,26．私の大好きなマルコビッチ様が、ただの悪役だったのがショック（&gt;＿&lt;）´´途...
1579,1580,イージー・ライダー,"156．《ネタバレ》 ステッペン・ウルフの""Born to be Wild""の曲がいいですね..."
1587,1588,イージー・ライダー,148．観る前の印象としては、「世間じゃ自由・平等などと騒いじゃいるが、俺たちにゃそんなこと...
1607,1608,イージー・ライダー,"128．音楽がよかった。""Born To Be Wild""が流れる冒頭のシーンは鳥肌もんでし..."


In [8]:
reviews_df = reviews_df.drop([21, 480,818,912,1163,1277,1295,1579,1587,1607,2306,2550,2784,4077,4249,4253,4263,4827,4878,4946,5483,5490,6436,6957,7148,7566,7739,7757,7776,7952])

In [9]:
reviews_df.ix[reviews_df['review'].str.contains('"')]

,id,title,review


In [ ]:
#con.close

次はレビューを読みだして形態素解析を行っていく。

### 2. レビューの形態素解析

##### 事前に作成しているテーブル

CREATE TABLE cnt_table(
    id int,
    title char(50),
    word char(30),
    num int
);

In [10]:
sql_query = '''
SELECT * FROM cnt_table
'''
words_df = sql_to_df(sql_query)

In [11]:
words_df.tail()

,id,title,word,num
874681,8352,グリフターズ／詐欺師たち,08,1
874682,8352,グリフターズ／詐欺師たち,:,1
874683,8352,グリフターズ／詐欺師たち,58,1
874684,8352,グリフターズ／詐欺師たち,）,1
874685,8352,グリフターズ／詐欺師たち,,1


In [ ]:
#con.close()

### 3. 特徴量行列の作成

In [12]:
i = 1
words_cnt = []

In [13]:
#The first step —————————————————————————————————————————————————
#全映画で使われたwordをとりだす。
sql_query = '''
SELECT word FROM cnt_table WHERE word not like '%''%' GROUP BY word ORDER BY sum(num) desc
'''
words_used_df = sql_to_df(sql_query)

In [14]:
words_used_df.tail(1)

,word
31566,￣*


In [15]:
31566 / 1900

16.613684210526316

In [62]:
query_all = []

n = 0

for num in range(1,17):     
    query = '''SELECT '''
    for item in range(len(words_used_df)):
        if n <= num*1900:
            #レビューIDごとに出現しているwordでカウントする。
            query += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{n}, '''.format(word=words_used_df.iloc[item, 0], n=n)
        
            n += 1
    
            if n == num*1900:
                break
    
    query += '''id FROM cnt_table GROUP BY id, title'''
    #print(query)
    query_all.append(query)

In [21]:
query = '''
SELECT id, title FROM cnt_table GROUP BY id, title
'''
id_title_df = sql_to_df(query)

In [18]:
query = '''
SELECT distinct id FROM cnt_table
'''
id_df = sql_to_df(query)

In [63]:
for num in range(1,17):
    print(num)
    res_df = sql_to_df(query_all[num-1])
    if num == 1:
        all_df = pd.merge(id_df, res_df, on='id')
    elif num > 1:
        all_df = pd.merge(all_df, res_df, on='id')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [64]:
all_df = pd.merge(all_df, id_title_df, on='id')

In [65]:
res_df = all_df.drop('id', axis=1)

In [66]:
res_df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a30391,a30392,a30393,a30394,a30395,a30396,a30397,a30398,a30399,title
0,3,4,6,2,2,3,4,3,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
1,5,4,5,4,3,3,3,3,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
2,5,6,5,6,4,3,4,1,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
3,3,1,2,1,3,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
4,4,5,2,3,5,2,2,3,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
5,3,0,1,3,1,0,4,1,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
6,3,3,3,3,3,0,6,1,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
7,3,2,2,3,4,0,2,1,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
8,3,3,1,3,4,1,2,0,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
9,3,3,1,2,0,1,1,2,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に


In [78]:
res_df.describe()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a30390,a30391,a30392,a30393,a30394,a30395,a30396,a30397,a30398,a30399
count,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,...,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000,8352.000000
mean,3.891164,3.736111,3.166188,2.538075,2.495570,2.320043,2.268918,2.136614,1.997246,1.681034,...,0.003592,0.003592,0.003592,0.003592,0.003592,0.003592,0.003592,0.003592,0.003592,0.003592
std,5.076779,3.492750,4.643813,3.029727,2.855139,3.136566,2.998263,2.820947,0.058865,0.745262,...,0.059829,0.061798,0.059829,0.061798,0.059829,0.061798,0.063706,0.059829,0.059829,0.059829
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,3.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,5.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,45.000000,36.000000,59.000000,26.000000,27.000000,30.000000,26.000000,27.000000,3.000000,4.000000,...,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000


In [ ]:
con.close

In [67]:
res_df.to_csv( 'tokuchouryou.csv' )

### 4. モデル作成

ここでは、UdemyのPython実践講座の Lec80-81 を参考にしてロジスティック回帰での多値分類を行う。

In [68]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [69]:
from sklearn import linear_model

参考 : http://qiita.com/yshi12/items/9f3f8ae69588da9f018f

In [70]:
res_df.target = res_df['title']

In [71]:
res_numpyMatrix_target = res_df.target.as_matrix()

In [72]:
res_df.data = res_df.iloc[ : , 0:30400]

In [73]:
res_numpyMatrix_data = res_df.iloc[ : , 0:30400].as_matrix()

In [74]:
# 説明変数をXに
X = res_numpyMatrix_data

#目的変数をYに
Y = res_numpyMatrix_target

In [75]:
#pandas.DataFrameにしておきましょう。

_data = DataFrame(X)

_target = DataFrame(Y,columns=['title'])

scikit-learnを使った多クラス分類


In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression()

# データを分割します。テストが全体の40%になるようにします。
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4,random_state=3)

# データを使って学習します。
logreg.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
# 精度を計算するのに便利なツールです。
from sklearn import metrics

# テストデータを予測します。
Y_pred = logreg.predict(X_test)

# 精度を計算してみましょう。
print(metrics.accuracy_score(Y_test,Y_pred))

0.360670457947


In [336]:
Y_test = Series(Y_test)
Y_pred = Series(Y_pred)

In [337]:
hikaku = []

In [338]:
hikaku = pd.concat([Y_test, Y_pred], axis=1)

In [339]:
hikaku

,0,1
0,愛は静けさの中に - みんなのシネマレビュー,愛は静けさの中に - みんなのシネマレビュー
1,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
2,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
3,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
4,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー - みんなのシネマレビュー
5,あなたに降る夢 - みんなのシネマレビュー,あなたに降る夢 - みんなのシネマレビュー
6,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー - みんなのシネマレビュー
7,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
8,あなたに降る夢 - みんなのシネマレビュー,あなたに降る夢 - みんなのシネマレビュー
9,アサシン(1993) - みんなのシネマレビュー,アサシン(1993) - みんなのシネマレビュー
